In [1]:
import optuna
import joblib
import numpy as np
import pandas as pd
import sys
from project_resources.import_utils import NotebookFinder
sys.meta_path.append(NotebookFinder())
from project_resources.cytochrome_P450 import HyperparamTuner, parse_jazzy_df

importing Jupyter notebook from C:\Users\Lukas\Documents\datacytochromy\project_resources\cytochrome_P450.ipynb


In [2]:
model_identifiers = ["linear", "KRR", "GB", "RF", "ANN"]
isozymes = ["3A4", "RLM", "HLC"]
data_splits = ["train", "test"]
splitters = ["rand", "scaff"]
rel_paths = {
    "3A4_source": r"project_resources/ChEMBL_3A4.csv",
    "3A4_sep": ";",
    "3A4": r"project_resources/3A4.csv",
    "3A4_train_scaff": r"project_resources/base_splits/scaffold_splitter/3A4_train.csv",
    "3A4_test_scaff": r"project_resources/base_splits/scaffold_splitter/3A4_test.csv",
    "3A4_train_rand": r"project_resources/base_splits/random/3A4_train.csv",
    "3A4_test_rand": r"project_resources/base_splits/random/3A4_test.csv",
    "3A4_jazzy_train_rand": r"project_resources/jazzy_splits/random/3A4_train.csv",
    "3A4_jazzy_test_rand": r"project_resources/jazzy_splits/random/3A4_test.csv",
    "3A4_jazzy_train_scaff": r"project_resources/jazzy_splits/scaffold_splitter/3A4_train.csv",
    "3A4_jazzy_test_scaff": r"project_resources/jazzy_splits/scaffold_splitter/3A4_test.csv",

    "RLM_source": r"project_resources/AID_1508591_datatable_all.csv",
    "RLM_sep": ",",
    "RLM": r"project_resources/RLM.csv",
    "RLM_train_scaff": r"project_resources/base_splits/scaffold_splitter/RLM_train.csv",
    "RLM_test_scaff": r"project_resources/base_splits/scaffold_splitter/RLM_test.csv",
    "RLM_train_rand": r"project_resources/base_splits/random/RLM_train.csv",
    "RLM_test_rand": r"project_resources/base_splits/random/RLM_test.csv",
    "RLM_jazzy_train_rand": r"project_resources/jazzy_splits/random/RLM_train.csv",
    "RLM_jazzy_test_rand": r"project_resources/jazzy_splits/random/RLM_test.csv",
    "RLM_jazzy_train_scaff": r"project_resources/jazzy_splits/scaffold_splitter/RLM_train.csv",
    "RLM_jazzy_test_scaff": r"project_resources/jazzy_splits/scaffold_splitter/RLM_test.csv",


    "HLC_source": r"project_resources/AID_1508603_datatable_all.csv",
    "HLC_sep": ",",
    "HLC": r"project_resources/HLC.csv",
    "HLC_train_scaff": r"project_resources/base_splits/scaffold_splitter/HLC_train.csv",
    "HLC_test_scaff": r"project_resources/base_splits/scaffold_splitter/HLC_test.csv",
    "HLC_train_rand": r"project_resources/base_splits/random/HLC_train.csv",
    "HLC_test_rand": r"project_resources/base_splits/random/HLC_test.csv",
    "HLC_jazzy_train_rand": r"project_resources/jazzy_splits/random/HLC_train.csv",
    "HLC_jazzy_test_rand": r"project_resources/jazzy_splits/random/HLC_test.csv",
    "HLC_jazzy_train_scaff": r"project_resources/jazzy_splits/scaffold_splitter/HLC_train.csv",
    "HLC_jazzy_test_scaff": r"project_resources/jazzy_splits/scaffold_splitter/HLC_test.csv"
}
# sampler - a method used to generate new sets of hyperparameters in each iteration of the optimization process
samplers = {
    'RandomSampler': optuna.samplers.RandomSampler,          # Sampler that selects hyperparameters randomly from the search space.
    'GridSampler': optuna.samplers.GridSampler,              # Sampler that performs a grid search over the hyperparameter space.
    'TPESampler': optuna.samplers.TPESampler,                # Sampler that uses a tree-structured Parzen estimator to model the objective function and sample new points from the search space.
    'CmaEsSampler': optuna.samplers.CmaEsSampler,            # Sampler that uses the Covariance Matrix Adaptation Evolution Strategy algorithm to efficiently search the hyperparameter space.
    'NSGAIISampler': optuna.samplers.NSGAIISampler,          # Multi-objective evolutionary algorithm that generates new samples using non-dominated sorting and crowding distance selection.
    'QMCSampler': optuna.samplers.QMCSampler,                # Quasi-Monte Carlo sampler that uses low-discrepancy sequences to sample the search space in a more efficient and evenly distributed way than random sampling.
    'BoTorchSampler': optuna.integration.BoTorchSampler,     # Sampler that leverages the BoTorch library for Bayesian optimization and can handle both continuous and categorical hyperparameters.
    'BruteForceSampler': optuna.samplers.BruteForceSampler,  # Sampler that exhaustively evaluates all possible combinations of hyperparameters in the search space.
}
# pruner - a technique used to eliminate unpromising trials during the course of hyperparameter optimization.
pruners = {
    'BasePruner': optuna.pruners.BasePruner,                            # This is the base class for all pruning strategies in Optuna. It provides a skeleton for implementing custom pruning strategies.
    'MedianPruner': optuna.pruners.MedianPruner,                        # A pruner that prunes unpromising trials that have median objective values, as determined in previous steps.
    'SuccessiveHalvingPruner': optuna.pruners.SuccessiveHalvingPruner,  # This pruner repeatedly splits trials into halves, discarding the lower performing half at each iteration.
    'HyperbandPruner': optuna.pruners.HyperbandPruner,                  # This pruner implements the Hyperband algorithm, which selects promising trials and runs them with different resource allocation schemes to determine the best one.
    'PercentilePruner': optuna.pruners.PercentilePruner,                # A pruner that prunes unpromising trials based on their percentile rank relative to all completed trials.
    'NopPruner': optuna.pruners.NopPruner,                              # A pruner that does nothing and does not prune any trials.
    'ThresholdPruner': optuna.pruners.ThresholdPruner,                  # This pruner prunes trials that have not reached a certain level of performance (i.e., objective value).
    'PatientPruner': optuna.pruners.PatientPruner,                      # This pruner prunes trials that do not show improvement over a certain number of steps (or epochs).
}
halflives = {}
mol_features = {}

In [3]:
# load Jazzy features from csv files
for splitter in splitters:
    print("\n")
    print(splitter)
    mol_features[splitter] = {}
    halflives[splitter] = {}
    for isozyme in isozymes:
        mol_features[splitter][isozyme] = {}
        halflives[splitter][isozyme] = {}
        for split in data_splits:
            print(isozyme, split)
            df = pd.read_csv(rel_paths[f"{isozyme}_jazzy_{split}_{splitter}"])
            features, thalfs, contains_nan = parse_jazzy_df(df)
            mol_features[splitter][isozyme][split] = features
            halflives[splitter][isozyme][split] = thalfs
            print(f"     {mol_features[splitter][isozyme][split][0]}")
            print(f"     {isozyme} mol_features {split} contains NaN: {contains_nan}")



rand
3A4 train
     56, [63, 'CC(C)(O)c1cc(F)c2c(c1)C(=O)N(Cc1ccc(Cl)cn1)[C@@]2(OCC1(O)CC1)c1ccc(Cl)cc1', 6.0, 13.1753, 1.4177, 11.3044, -7.3559, -125.8906, -119.4889]
     [  13.1753    1.4177   11.3044   -7.3559 -125.8906 -119.4889]
     3A4 mol_features train contains NaN: False
3A4 test
     14, [23, 'Cc1ncsc1-c1ccc([C@H](CC(=O)NCCCCCCNC(=O)COc2c(-c3csc(N4CCOCC4)n3)ccc(F)c2F)NC(=O)[C@@H]2C[C@@H](O)CN2C(=O)[C@@H](NC(=O)C2(F)CC2)C(C)(C)C)cc1', 0.3767, 28.4327, 3.274, 23.4872, -4.0348, -286.1545, -257.0326]
     [  28.4327    3.274    23.4872   -4.0348 -286.1545 -257.0326]
     3A4 mol_features test contains NaN: False
RLM train
     removed index 1047 corresponding to NaN
     removed index 1517 corresponding to NaN
     2022, [158, 'O=c1cc(N2CCOCC2)oc2c1ccc1ccccc12', 30.0, 8.0486, 0.0, 5.2165, -13.5913, -63.7029, -67.7504]
     [  8.0486   0.       5.2165 -13.5913 -63.7029 -67.7504]
     RLM mol_features train contains NaN: False
RLM test
     removed index 174 corresponding to Na

In [5]:
sampler = samplers['TPESampler']
pruner = pruners["BasePruner"]
n_trials = 200
for splitter in splitters:
    if splitter == "rand":
        splitter_name = "random"
    else:
        splitter_name = "scaffold_splitter"

    for isozyme in isozymes:
        X_train = mol_features[splitter][isozyme]["train"]
        y_train = np.log(halflives[splitter][isozyme]["train"])  # natural log half-lives -> values are less spread out
        X_test = mol_features[splitter][isozyme]["test"]
        y_test = np.log(halflives[splitter][isozyme]["test"])
        for model_identifier in model_identifiers:
            print(splitter_name, isozyme, model_identifier)
            study = optuna.create_study(study_name=model_identifier, directions=['minimize'], pruner=pruner,
                                        storage=f"sqlite:///project_resources/optuna/jazzy/{splitter_name}/{isozyme}/db.{model_identifier}", load_if_exists=True)
            test = HyperparamTuner(model_identifier, X_train, y_train, X_test, y_test)
            study.optimize(test.objective, n_trials=n_trials, n_jobs=-1)  # catch=(ValueError,)
            joblib.dump(study, f"./project_resources/optuna/jazzy/{splitter_name}/{isozyme}/{model_identifier}.pkl")

[I 2023-10-11 06:59:12,987] Using an existing study with name 'linear' instead of creating a new one.


random 3A4 linear


[I 2023-10-11 06:59:13,671] Trial 8 finished with value: 1.4055792338666269 and parameters: {'alpha': 0.017591569877912345, 'l1_ratio': 0.964680833116307}. Best is trial 0 with value: 1.4038627192898365.
[I 2023-10-11 06:59:13,937] Trial 11 finished with value: 1.361241861694033 and parameters: {'alpha': 0.0812355109698709, 'l1_ratio': 0.08450223915968114}. Best is trial 11 with value: 1.361241861694033.
[I 2023-10-11 06:59:14,092] Trial 7 finished with value: 1.4178962175797631 and parameters: {'alpha': 0.009864948436884355, 'l1_ratio': 0.9901756813079965}. Best is trial 11 with value: 1.361241861694033.
[I 2023-10-11 06:59:14,343] Trial 14 finished with value: 1.3812513765516237 and parameters: {'alpha': 0.08910494275045223, 'l1_ratio': 0.5135890752430664}. Best is trial 11 with value: 1.361241861694033.
[I 2023-10-11 06:59:14,344] Trial 13 finished with value: 1.4122052683532835 and parameters: {'alpha': 0.07428471558688238, 'l1_ratio': 0.15148988414336717}. Best is trial 11 with va

random 3A4 KRR


[I 2023-10-11 06:59:41,765] Trial 13 finished with value: 1.453434684705731 and parameters: {'alpha': 0.833868652227199, 'gamma': 6.1920434330254446e-15, 'kernel': 'rbf'}. Best is trial 9 with value: 1.376115276011723.
[I 2023-10-11 06:59:41,774] Trial 9 finished with value: 1.376115276011723 and parameters: {'alpha': 0.7868905362344104, 'gamma': 5.224902974359913e-15, 'kernel': 'rbf'}. Best is trial 9 with value: 1.376115276011723.
[I 2023-10-11 06:59:41,886] Trial 5 finished with value: 1.3840306290217816 and parameters: {'alpha': 0.2255045958969689, 'gamma': 6.081953865800383e-15, 'kernel': 'linear'}. Best is trial 9 with value: 1.376115276011723.
[I 2023-10-11 06:59:41,983] Trial 6 finished with value: 1.4109942622308378 and parameters: {'alpha': 0.9639486790426622, 'gamma': 7.32624119186187e-16, 'kernel': 'rbf'}. Best is trial 9 with value: 1.376115276011723.
[I 2023-10-11 06:59:42,011] Trial 3 finished with value: 1.3867713855700263 and parameters: {'alpha': 0.4851231136817665, '

...

[I 2023-10-11 09:36:17,139] Trial 13 finished with value: 0.48184267592507696 and parameters: {'n_estimators': 20, 'learning_rate': 0.21174785612696173, 'max_depth': 3}. Best is trial 1 with value: 0.4756038094216188.
[I 2023-10-11 09:36:17,308] Trial 3 finished with value: 0.46605549050083095 and parameters: {'n_estimators': 50, 'learning_rate': 0.5242148467570048, 'max_depth': 4}. Best is trial 3 with value: 0.46605549050083095.
[I 2023-10-11 09:36:18,094] Trial 11 finished with value: 0.49355829998698253 and parameters: {'n_estimators': 200, 'learning_rate': 0.05677881541182676, 'max_depth': 3}. Best is trial 3 with value: 0.46605549050083095.
[I 2023-10-11 09:36:18,364] Trial 12 finished with value: 0.4655988010298464 and parameters: {'n_estimators': 10, 'learning_rate': 0.9528752836361396, 'max_depth': 2}. Best is trial 12 with value: 0.4655988010298464.
[I 2023-10-11 09:36:18,378] Trial 8 finished with value: 0.4811918976721066 and parameters: {'n_estimators': 20, 'learning_rate'

scaffold_splitter HLC RF


[I 2023-10-11 09:38:17,692] Trial 4 finished with value: 0.44587166608541434 and parameters: {'n_estimators': 50, 'max_features': 'log2', 'max_depth': 4}. Best is trial 4 with value: 0.44587166608541434.
[I 2023-10-11 09:38:18,267] Trial 5 finished with value: 0.4537877903893184 and parameters: {'n_estimators': 20, 'max_features': 'auto', 'max_depth': 4}. Best is trial 4 with value: 0.44587166608541434.
[I 2023-10-11 09:38:18,810] Trial 9 finished with value: 0.4674159276758301 and parameters: {'n_estimators': 20, 'max_features': 'sqrt', 'max_depth': 5}. Best is trial 4 with value: 0.44587166608541434.
[I 2023-10-11 09:38:18,872] Trial 8 finished with value: 0.46956800727132086 and parameters: {'n_estimators': 50, 'max_features': 'auto', 'max_depth': 3}. Best is trial 4 with value: 0.44587166608541434.
[I 2023-10-11 09:38:18,983] Trial 12 finished with value: 0.4737672949583082 and parameters: {'n_estimators': 200, 'max_features': 'auto', 'max_depth': 5}. Best is trial 4 with value: 0.

scaffold_splitter HLC ANN


[I 2023-10-11 09:44:39,941] Trial 11 finished with value: 3.165180089393108 and parameters: {'learning_rate_init': 0.039937546710066676, 'hidden_layer_sizes': [20, 20]}. Best is trial 1 with value: 2.572426858100426.
[I 2023-10-11 09:44:44,308] Trial 8 finished with value: 2.480471711161741 and parameters: {'learning_rate_init': 0.031787266108631224, 'hidden_layer_sizes': [50]}. Best is trial 8 with value: 2.480471711161741.
[I 2023-10-11 09:44:44,937] Trial 13 finished with value: 2.5675879557894756 and parameters: {'learning_rate_init': 0.07006039827925171, 'hidden_layer_sizes': [50, 50]}. Best is trial 8 with value: 2.480471711161741.
[I 2023-10-11 09:44:45,076] Trial 6 finished with value: 2.5629325131055203 and parameters: {'learning_rate_init': 0.023531691896758335, 'hidden_layer_sizes': [20]}. Best is trial 8 with value: 2.480471711161741.
[I 2023-10-11 09:44:47,686] Trial 5 finished with value: 2.6828633607301473 and parameters: {'learning_rate_init': 0.004503132535918747, 'hid